In [1]:
import sys
import warnings
import concurrent.futures
import queue
import numpy as np
import time
import pandas as pd
import atexit
from time import time, strftime, localtime
from datetime import timedelta
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

np.set_printoptions(suppress=True)
np.warnings.filterwarnings('ignore')
np.set_printoptions(threshold=sys.maxsize)
from matplotlib.pyplot import figure


def ACC_CMPC(i,j):
    ###################################################################################

    def share(input):
        # print(input)
        rs1 = np.random.RandomState(10)
        x = 0.00000001 * rs1.random((np.shape(input)))

        rs2 = np.random.RandomState(20)
        y = 0.00000001 * rs2.random((np.shape(input)))

        z = input - x - y

        return x, y, z

    def sharey(input):
        np.random.seed(1)
        x = np.random.randint(0, 2, np.shape(input))

        np.random.seed(1)
        y = np.random.randint(0, 2, np.shape(input))

        z = input - x - y

        return x, y, z

    ###################################################################################

    sizex = i
    sizey = j

    rs12 = np.random.RandomState(1)
    X_train = np.round(rs12.uniform(-1, 1, (sizex, sizey)), 7)
    X_train1, X_train2, X_train3 = share(X_train)

    rs13 = np.random.RandomState(2)
    y_train = rs13.randint(0, 2, (sizex, 1))
    y_train1, y_train2, y_train3 = sharey(y_train)

    ###################################################################################

    #     data = pd.read_csv('lbw.csv')

    #     Y1 = data.iloc[:, -1:]
    #     y = Y1.values.reshape(np.shape(data)[0], 1)

    #     data.drop(data.columns[len(data.columns) - 1], axis=1, inplace=True)
    #     X = np.array(data)

    #     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

    #     X_train1 , X_train2 , X_train3 = share(X_train)
    #     y_train1 , y_train2 , y_train3 = sharey(y_train)

    ###################################################################################

    def CP1(beta):

        def mul_multiply(first, second):

            rs1 = np.random.RandomState(1)
            r12 = rs1.uniform(0.0000000001, 10 ** (-40), np.shape(first))

            rs2 = np.random.RandomState(2)
            j12 = rs2.uniform(0.0000000001, 10 ** (-40), np.shape(second))
            # print(r12)

            q1.put(r12)
            q1.put(j12)

            r31 = q3.get()
            j31 = q3.get()

            XA11 = first + r12 - r31

            XB11 = second + j12 - j31

            q11.put(XA11)
            q11.put(XB11)

            XA33 = q33.get()
            XB33 = q33.get()

            W1 = XA11 * XB11 + XA11 * XB33 + XA33 * XB11
            # print(W1)

            return W1

        def mul_multiply_matrix_full(first, second):
            # print(first)

            rs3 = np.random.RandomState(3)
            r12 = 0.01 * rs3.random((np.shape(first)))
            # print(r12)
            rs4 = np.random.RandomState(4)
            j12 = 0.01 * rs4.random((np.shape(second)))

            q1.put(r12)
            q1.put(j12)

            r31 = q3.get()
            j31 = q3.get()

            XA11 = first + r12 - r31
            # print(XA11)

            XB11 = second + j12 - j31
            # print(j31)

            q11.put(XA11)
            q11.put(XB11)

            XA33 = q33.get()
            XB33 = q33.get()

            W1 = XA11.dot(XB11) + XA11.dot(XB33) + XA33.dot(XB11)

            return W1

        def mul_sharemind_matrix_direct(first, second):
            # print(second)

            rs5 = np.random.RandomState(5)
            r12 = rs5.uniform(0.0000000001, 10 ** (-40), np.shape(first))
            rs6 = np.random.RandomState(6)
            j12 = rs6.uniform(0.0000000001, 10 ** (-40), np.shape(second))

            q1.put(r12)
            q1.put(j12)

            r31 = q3.get()
            j31 = q3.get()

            XA11 = first + r12 - r31
            XB11 = second + j12 - j31

            q11.put(XA11)
            q11.put(XB11)

            XA33 = q33.get()
            XB33 = q33.get()

            W1 = np.multiply(XA11, XB11) + np.multiply(XA11, XB33) + np.multiply(XA33, XB11)
            return W1

        X = X_train1
        y = y_train1

        H_temp = mul_multiply_matrix_full(X.T, X)

        c = 0.0000002
        q = np.size(H_temp, 0)
        I = np.identity(q)
        Xs = mul_multiply(c, I / 3)
        Ms = mul_multiply(c, H_temp)

        for x in range(0, 25):
            Ms_Xs = mul_multiply_matrix_full(Xs, Ms)
            mul_2 = mul_multiply([[2 / 3]], Xs)
            X_s_t = mul_2 - Ms_Xs

            Ms_Ms = mul_multiply_matrix_full(Ms, Ms)
            Ms_2 = mul_multiply([[2 / 3]], Ms)
            M_s_t = Ms_2 - Ms_Ms

            Xs = X_s_t
            Ms = M_s_t

        M_s_t_temp = M_s_t

        Hessian_Invers_TEMP = X_s_t

        Hessian_Invers = -4 * Hessian_Invers_TEMP

        for i in range(0, 20):

            beta_old = beta

            np.random.seed(1)
            XB = mul_multiply_matrix_full(X, beta)

            Pi_temp = np.exp(-XB)

            Pi_temp11, Pi_temp12, Pi_temp13 = share(Pi_temp)

            q12.put(Pi_temp12)
            q13.put(Pi_temp13)

            Pi_temp21 = q31.get()
            Pi_temp31 = q21.get()

            e_to_the_XB_temp = mul_sharemind_matrix_direct(Pi_temp11, Pi_temp21)

            e_to_the_XB = mul_sharemind_matrix_direct(e_to_the_XB_temp, Pi_temp31) + 0.4

            np.random.seed(1)
            c_0_1 = np.random.uniform(0.01, 10 ** (-5), np.shape(e_to_the_XB))
            Xs = c_0_1 * 1 / 3
            Ms = mul_sharemind_matrix_direct(c_0_1 * 1 / 3, e_to_the_XB)

            for x in range(0, 20):
                Ms_Xs = mul_sharemind_matrix_direct(Xs, Ms)
                # print(Ms)
                mul_2 = mul_multiply([[2 / 3]], Xs)
                X_s_t = mul_2 - Ms_Xs

                Ms_Ms = mul_sharemind_matrix_direct(Ms, Ms)
                Ms_2 = mul_multiply([[2 / 3]], Ms)
                M_s_t = Ms_2 - Ms_Ms

                Xs = X_s_t
                Ms = M_s_t

            grad_temp = y - X_s_t

            Gradient = mul_multiply_matrix_full(X.T, grad_temp)
            step = mul_multiply_matrix_full(Hessian_Invers, Gradient)

            beta = beta_old - step

        return beta

    ###################################################################################

    def CP2(beta):
        def mul_multiply(first, second):

            rs1 = np.random.RandomState(1)
            r23 = rs1.uniform(0.0000000001, 10 ** (-40), np.shape(first))

            rs2 = np.random.RandomState(2)
            j23 = rs2.uniform(0.0000000001, 10 ** (-40), np.shape(second))
            # print(j23)

            q2.put(r23)
            q2.put(j23)

            r12 = q1.get()
            j12 = q1.get()

            XA22 = first + r23 - r12
            XB22 = second + j23 - j12

            q22.put(XA22)
            q22.put(XB22)

            XA11 = q11.get()
            XB11 = q11.get()

            W2 = XA22 * (XB22) + XA22 * (XB11) + XA11 * (XB22)
            return W2

        def mul_multiply_matrix_full(first, second):
            # print(second)
            rs3 = np.random.RandomState(3)
            r23 = 0.01 * rs3.random((np.shape(first)))

            rs4 = np.random.RandomState(4)
            j23 = 0.01 * rs4.random((np.shape(second)))

            q2.put(r23)
            q2.put(j23)

            r12 = q1.get()
            j12 = q1.get()

            XA22 = first + r23 - r12
            XB22 = second + j23 - j12

            q22.put(XA22)
            q22.put(XB22)

            XA11 = q11.get()
            XB11 = q11.get()

            W2 = XA22.dot(XB22) + XA22.dot(XB11) + XA11.dot(XB22)
            return W2

        def mul_sharemind_matrix_direct(first, second):

            rs5 = np.random.RandomState(5)
            r23 = rs5.uniform(00.0000000001, 10 ** (-40), np.shape(first))

            rs6 = np.random.RandomState(6)
            j23 = rs6.uniform(0.0000000001, 10 ** (-40), np.shape(second))

            q2.put(r23)
            q2.put(j23)

            r12 = q1.get()
            j12 = q1.get()

            XA22 = first + r23 - r12
            XB22 = second + j23 - j12

            q22.put(XA22)
            q22.put(XB22)

            XA11 = q11.get()
            XB11 = q11.get()

            W2 = np.multiply(XA22, XB22) + np.multiply(XA22, XB11) + np.multiply(XA11, XB22)
            return W2

        X = X_train2
        y = y_train2

        H_temp = mul_multiply_matrix_full(X.T, X)

        c = 0.0000002
        q = np.size(H_temp, 0)
        I = np.identity(q)
        Xs = mul_multiply(c, I / 3)
        Ms = mul_multiply(c, H_temp)

        for x in range(0, 25):
            Ms_Xs = mul_multiply_matrix_full(Xs, Ms)
            mul_2 = mul_multiply([[2 / 3]], Xs)
            X_s_t = mul_2 - Ms_Xs

            Ms_Ms = mul_multiply_matrix_full(Ms, Ms)
            Ms_2 = mul_multiply([[2 / 3]], Ms)
            M_s_t = Ms_2 - Ms_Ms

            Xs = X_s_t
            Ms = M_s_t

        M_s_t_temp = M_s_t

        Hessian_Invers_TEMP = X_s_t

        Hessian_Invers = -4 * Hessian_Invers_TEMP

        for i in range(0, 20):

            beta_old = beta

            np.random.seed(1)
            XB = mul_multiply_matrix_full(X, beta)

            Pi_temp = np.exp(-XB)

            Pi_temp21, Pi_temp22, Pi_temp23 = share(Pi_temp)

            q21.put(Pi_temp21)
            q23.put(Pi_temp23)

            Pi_temp12 = q12.get()
            Pi_temp32 = q32.get()

            e_to_the_XB_temp = mul_sharemind_matrix_direct(Pi_temp22, Pi_temp12)

            e_to_the_XB = mul_sharemind_matrix_direct(e_to_the_XB_temp, Pi_temp32) + 0.3

            np.random.seed(1)
            c_0_2 = np.random.uniform(0.0000000001, 10 ** (-40), np.shape(e_to_the_XB))
            Xs = c_0_2 * 1 / 3
            Ms = mul_sharemind_matrix_direct(c_0_2 * 1 / 3, e_to_the_XB)

            for x in range(0, 20):
                Ms_Xs = mul_sharemind_matrix_direct(Xs, Ms)
                mul_2 = mul_multiply([[2 / 3]], Xs)
                X_s_t = mul_2 - Ms_Xs

                Ms_Ms = mul_sharemind_matrix_direct(Ms, Ms)
                Ms_2 = mul_multiply([[2 / 3]], Ms)
                M_s_t = Ms_2 - Ms_Ms

                Xs = X_s_t
                Ms = M_s_t

            grad_temp = y - X_s_t

            Gradient = mul_multiply_matrix_full(X.T, grad_temp)

            step = mul_multiply_matrix_full(Hessian_Invers, Gradient)

            beta = beta_old - step

        return beta

    ###################################################################################

    def CP3(beta):
        def mul_multiply(first, second):

            rs1 = np.random.RandomState(1)
            r31 = rs1.uniform(0.0000000001, 10 ** (-40), np.shape(first))
            rs2 = np.random.RandomState(2)
            j31 = rs2.uniform(0.0000000001, 10 ** (-40), np.shape(second))

            q3.put(r31)
            q3.put(j31)

            r23 = q2.get()
            j23 = q2.get()

            XA33 = first + r31 - r23
            XB33 = second + j31 - j23

            q33.put(XA33)
            q33.put(XB33)

            XA22 = q22.get()
            XB22 = q22.get()

            W3 = XA33 * (XB33) + XA33 * (XB22) + XA22 * (XB33)
            return W3

        def mul_multiply_matrix_full(first, second):

            rs3 = np.random.RandomState(3)
            r31 = 0.01 * rs3.random((np.shape(first)))

            rs4 = np.random.RandomState(4)
            j31 = 0.01 * rs4.random((np.shape(second)))

            q3.put(r31)
            q3.put(j31)

            r23 = q2.get()
            j23 = q2.get()

            XA33 = first + r31 - r23
            XB33 = second + j31 - j23

            q33.put(XA33)
            q33.put(XB33)

            XA22 = q22.get()
            XB22 = q22.get()

            W3 = XA33.dot(XB33) + XA33.dot(XB22) + XA22.dot(XB33)
            return W3

        def mul_sharemind_matrix_direct(first, second):

            rs5 = np.random.RandomState(5)
            r31 = rs5.uniform(0.0000000001, 10 ** (-40), np.shape(first))
            rs6 = np.random.RandomState(6)
            j31 = rs6.uniform(0.0000000001, 10 ** (-40), np.shape(second))

            q3.put(r31)
            q3.put(j31)

            r23 = q2.get()
            j23 = q2.get()

            XA33 = first + r31 - r23
            XB33 = second + j31 - j23

            q33.put(XA33)
            q33.put(XB33)

            XA22 = q22.get()
            XB22 = q22.get()

            W3 = np.multiply(XA33, XB33) + np.multiply(XA33, XB22) + np.multiply(XA22, XB33)
            return W3

        X = X_train3
        y = y_train3

        H_temp = mul_multiply_matrix_full(X.T, X)

        c = 0.0000002
        q = np.size(H_temp, 0)
        I = np.identity(q)
        Xs = mul_multiply(c, I / 3)
        Ms = mul_multiply(c, H_temp)

        for x in range(0, 25):
            Ms_Xs = mul_multiply_matrix_full(Xs, Ms)
            mul_2 = mul_multiply([[2 / 3]], Xs)
            X_s_t = mul_2 - Ms_Xs

            Ms_Ms = mul_multiply_matrix_full(Ms, Ms)
            Ms_2 = mul_multiply([[2 / 3]], Ms)
            M_s_t = Ms_2 - Ms_Ms

            Xs = X_s_t
            Ms = M_s_t

        M_s_t_temp = M_s_t

        Hessian_Invers_TEMP = X_s_t

        Hessian_Invers = -4 * Hessian_Invers_TEMP

        for i in range(0, 20):

            beta_old = beta

            XB = mul_multiply_matrix_full(X, beta)
            # print(XB)

            Pi_temp = np.exp(-XB)
            # print(Pi_temp)

            Pi_temp31, Pi_temp32, Pi_temp33 = share(Pi_temp)

            q31.put(Pi_temp31)
            q32.put(Pi_temp32)

            Pi_temp13 = q13.get()
            Pi_temp23 = q23.get()

            e_to_the_XB_temp = mul_sharemind_matrix_direct(Pi_temp33, Pi_temp13)
            e_to_the_XB = mul_sharemind_matrix_direct(e_to_the_XB_temp, Pi_temp23) + 0.3

            np.random.seed(1)
            c_0_3 = np.random.uniform(0.001, 10 ** (-40), np.shape(e_to_the_XB))
            Xs = c_0_3 * 1 / 3
            Ms = mul_sharemind_matrix_direct(c_0_3 * 1 / 3, e_to_the_XB)

            for x in range(0, 20):
                Ms_Xs = mul_sharemind_matrix_direct(Xs, Ms)
                mul_2 = mul_multiply([[2 / 3]], Xs)
                X_s_t = mul_2 - Ms_Xs

                Ms_Ms = mul_sharemind_matrix_direct(Ms, Ms)
                Ms_2 = mul_multiply([[2 / 3]], Ms)
                M_s_t = Ms_2 - Ms_Ms

                Xs = X_s_t
                Ms = M_s_t

            grad_temp = y - X_s_t

            Gradient = mul_multiply_matrix_full(X.T, grad_temp)

            step = mul_multiply_matrix_full(Hessian_Invers, Gradient)

            beta = beta_old - step

        return beta

    ###################################################################################

    q1 = queue.Queue()
    q2 = queue.Queue()
    q3 = queue.Queue()

    q11 = queue.Queue()
    q22 = queue.Queue()
    q33 = queue.Queue()

    q12 = queue.Queue()
    q13 = queue.Queue()

    q21 = queue.Queue()
    q23 = queue.Queue()

    q31 = queue.Queue()
    q32 = queue.Queue()

    ###################################################################################

    import atexit
    from time import time, strftime, localtime
    from datetime import timedelta

    def ACMPC():

        beta = np.zeros((np.size(X_train, 1), 1))

        beta1, beta2, beta3 = share(beta)

        with concurrent.futures.ThreadPoolExecutor() as executor:
            f1 = executor.submit(CP1, beta1)
            f2 = executor.submit(CP2, beta2)
            f3 = executor.submit(CP3, beta3)

            Total = (f1.result() + f2.result() + f3.result())

            #         print('Beta1, round' , iter_count , '\n', f1.result(), '\n')

            #         print('Beta2, round' , iter_count, '\n', f2.result(), '\n')

            #         print('Beta3, round' , iter_count, '\n', f3.result(), '\n')

            #         print('Tot, round' , iter_count, '\n', Total, '\n')

            #         print(is_pos_def(Total))

            #         print('######################' ,'\n')

            return Total

        return ret

    start = time()

    beta = ACMPC()

    end = time()
    elapsed = end - start
#     print('Elapsed :', end - start)
#     print('\n ACC_CMPC: \n', beta)
    return elapsed


def mem1():

    beta1 = ACC_CMPC(50000,200)  
    
#     print(beta1)
    
    return beta1


global m

m = 0

from memory_profiler import memory_usage

for x in range(0, 10): 

    mem = max(memory_usage(proc=mem1))
    
    m  = m + mem

    print("Maximum memory used: {} MiB".format(mem))
    
print(m/10)

Maximum memory used: 5688.671875 MiB
Maximum memory used: 5732.640625 MiB
Maximum memory used: 5759.578125 MiB
Maximum memory used: 5763.5625 MiB
Maximum memory used: 5777.25 MiB
Maximum memory used: 5782.015625 MiB
Maximum memory used: 5794.265625 MiB
Maximum memory used: 5795.046875 MiB
Maximum memory used: 5806.109375 MiB
Maximum memory used: 5808.21875 MiB
5770.7359375
